Code to view the pulses kept alongside those cut away.

In [1]:
import sys
import time
import matplotlib.pyplot as plt # https://matplotlib.org/
import numpy as np # http://www.numpy.org/
import tekwfm
import os
from os.path import exists
from scipy import signal
from scipy import integrate
from scipy.optimize import curve_fit
from scipy.special import gamma
import matplotlib.cm as cm
from collections import Counter

import gc
import re
import tempfile
from collections import defaultdict, namedtuple
from pathlib import Path
import pandas as pd

In [2]:
doVerbose = False

#import the wfm file
filename_wfm = "PMT4_AllAnalysis/PMT4/220126_darkrate_1750_10000_3_on"

#import the cleaned file and make data frame
filename_clean = pd.read_csv("PMT4_AllAnalysis/PMT4Cleaned/220126_darkrate_1750_10000_3_on_processed_threshold11_cleaned.csv")


#choose number of events to be seen and starting event (first event is event 0)
startEvent = 600
nEvents = 100

volts, tstart, tscale, tfrac, tdatefrac, tdate = tekwfm.read_wfm(filename_wfm+".wfm")
if doVerbose:
    print('>>>>', volts, tstart, tscale, tfrac, tdatefrac, tdate)

samples = volts.shape

print(len(volts))
print(tscale)
print('samples', samples[0])
tstop = samples[0]*tscale+tstart
sampleTimes = [tstart+x*tscale for x in range(samples[0])]


#the clean file is the list of all of the events which have passed the cuts, and should be good pulses
wfm_index_col = filename_clean.columns[0]
passed_events = set(filename_clean[wfm_index_col].astype(int).values)
print(f"Number of passing events in CSV: {len(passed_events)}")


#plot setup:
bad_events = []
good_events = []

for event in range(startEvent, startEvent + nEvents):
    if event in passed_events:
        good_events.append(event)
    else:
        bad_events.append(event)

print("Bad events:", bad_events)
print("Good events:", good_events)


n_bad = len(bad_events)
n_good = len(good_events)
n_rows = max(n_bad, n_good)

fig, axes = plt.subplots(nrows=n_rows, ncols=2, figsize=(12, 6*n_rows))

# Fix shape if only one row
if n_rows == 1:
    axes = np.array([axes])


for i, event in enumerate(bad_events):

    tempVolt = volts[:, event]

    ax = axes[i, 0]

    ax.plot(np.array(sampleTimes)*1e9, np.array(tempVolt)*1e3, color='black')
    ax.tick_params(axis='both', labelsize=12)
    ax.set_xlabel('Oscilloscope Timebase (ns)', fontsize=14)
    ax.set_xlim(-20,80)
    ax.set_ylabel('Voltage (mV)', fontsize=14)
    ax.set_ylim(-25, 2)

    ax.set_title(f"Event {event} — FAILED", fontsize=12)

    del(tempVolt)


for i, event in enumerate(good_events):

    tempVolt = volts[:, event]

    ax = axes[i, 1]

    ax.plot(np.array(sampleTimes)*1e9, np.array(tempVolt)*1e3, color='black')
    ax.tick_params(axis='both', labelsize=12)
    ax.set_xlabel('Oscilloscope Timebase (ns)', fontsize=14)
    ax.set_xlim(-20,80)
    ax.set_ylabel('Voltage (mV)', fontsize=14)
    ax.set_ylim(-25, 2)

    ax.set_title(f"Event {event} — PASSED", fontsize=12)

    del(tempVolt)


for i in range(n_rows):
    if i >= n_bad:
        axes[i, 0].axis('off')
    if i >= n_good:
        axes[i, 1].axis('off')


fig.text(0.25, 0.97, "BAD pulses (Failed the cuts)", ha='center', va='top',
         fontsize=15, color='red')

fig.text(0.75, 0.97, "GOOD pulses (Passed the cuts)", ha='center', va='top',
         fontsize=15, color='green')
plt.tight_layout(rect=[0, 0, 1, 0.96]) 
plt.show()

1250
8e-11
samples 1250
Number of passing events in CSV: 1632
Bad events: [602, 603, 605, 616, 633, 635, 647, 648, 652, 657, 669, 672, 675, 676, 679, 681, 686, 689, 691, 692, 693, 699]
Good events: [600, 601, 604, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 634, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 649, 650, 651, 653, 654, 655, 656, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 670, 671, 673, 674, 677, 678, 680, 682, 683, 684, 685, 687, 688, 690, 694, 695, 696, 697, 698]
